사이킷런의 model_selection 모듈은 학습 데이터와 테스트 데이터 세트를 분리하거나 교차 검증 분할 및 평가, 그리고 Estimator의 하이퍼 파라미터를
튜닝하기 위한 다양한 함수와 클래스를 제공.

먼저 앞의 예제에서도 소개되었지만, 전체 데이터를 학습 데이터와 테스트 데이터 세트로 분리해주는 train_test_split()부터 자세히 살펴보겠음

### 학습/테스트 데이터 세트 분리 - train_test_split()

먼저 테스트 데이터 세트를 이용하지 않고 학습 데이터 세트로만 학습하고 예측하면 무엇이 문제인지 살펴보겠음.

다음 예제는 학습과 예측을 동일한 데이터 세트로 수행한 결과

In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()
dt_clf = DecisionTreeClassifier()

train_data = iris.data
train_label = iris.target

dt_clf.fit(train_data, train_label)

# 학습 데이터 세트로 예측 수행
pred = dt_clf.predict(train_data)
print(' 예측 정확도 : ', accuracy_score(train_label, pred))

 예측 정확도 :  1.0


에측 정확도가 1.0 (100%)인 이유는 학습 데이터와 테스트 데이터를 같이 썻기에 가능한일, 
쉽게 이야기하자면 이미 정답지를 보고난 뒤에 정답을 맞추는 것으로 예시를 들 수 있음.

따라서, 데이터 내에서 학습데이터와 테스트 데이터를 분리하여 쓰는 것이 좋으며 train_test_split() 를 이용하여 쉽게 분류가 가능함

먼저 sklearn.model_selection 모듈에서 train_test_split을 로드함.

train_test_split()는 첫 번째 파라미터로 피처 데이터 세트, 두 번째 파라미터로 레이블 데이터 세트를 입력받음.
그리고 선택적으로 다음 파라미터를 입력받음.

test_size : 전체 데이터에서 테스트 데이터 세트 크기를 얼마로 샘플링할 것인가를 결정, 디폴트 0.25, 즉 25%

train_size : 전체 데이터에서 학습용 데이터 세트 크기를 얼마로 샘플링할 것인가를 결정. test_size parameter을 통상적으로 사용하기 때문에 train_size는 잘 사용되지 않음

shuffle : 테이터를 분리하기 전에 데이터를 미리 섞을지를 결정함. 디폴트는 True임. 데이터를 분산시켜서 좀 더 호율적인 학습 및 테스트 데이터 세트를 만드는데 사용됨

random_state : random_state는 호출할 때마다 동일한 학습/테스트용 데이터 세트를 생성하기 위해 주어지는 난수 값임. train_test_split()는 호출 시 무작위로 데이터를 분리하므로 random_state를 지정하기 않으면 수행할 때마다 다른 학습/테스트 용 데이터를 생성.
이 책에서 소개하는 예제에는 실습용 예제이므로 수행할 때마다 동일한 데이터 세트로 분리하기 위해 random_state를 일정한 숫자 값으로 부여

train_test_split()의 반환값은 튜플 형태. 순차적으로 학습용 데이터의 피처 데이터 세트, 테스트용 데이터의 피처 데이터 세트, 학습용 데이터의 레이블 데이터 세트, 테스트용 데이터의 레이블 데이터 세트가 반환됨

붓꽃 데이터 세트를 train_test_split()을 이용해 테스트 데이터 세트를 전체의 30%로, 학습 데이터 세트를 70%로 분리.
앞의 예제와는 다르게 random_state=121로 변경해 데이터 세트를 변환 시켜봄

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

dt_clf = DecisionTreeClassifier()
iris_data = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=0.3, random_state=121)

학습 데이터를 기반으로 DecisionTreeClassifier를 학습하고 이 모델을 이용해 예측 정확도를 측정

In [7]:
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('에측 정확도: {0:4f}'.format(accuracy_score(y_test, pred)))

에측 정확도: 0.955556


예측 정확도는 95% 정도 이다. 
붓꽃 데이터는 150개의 데이터로 데이터 양이 크지 않아 전체의 30% 정도인 테스트 데이터는 45개 정도밖에 되지 않으므로 이를 통해 알고리즘의 예측 성능을 판단하기에는 그리 적절하지 않음.
학습을 위한 데이터의 양을 일정 수준 이상으로 보장하는 것도 중요하지만, 학습된 모델에 대해 다양한 데이터를 기반으로 예측 성능을 평가해 보는 것도 매우 중요

## 교차검증

